# 📊 Análise de E-commerce: O que faz um produto ser 5 estrelas?
**Autor:** Guilherme Pires Oliveira
**Tecnologias:** Python, Pandas, Seaborn, NLP (Processamento de Linguagem Natural)

---

### 🎯 Objetivo do Projeto
Analisar dados reais de vendas de eletrônicos na Amazon para responder a perguntas estratégicas:
1. Preço alto é sinônimo de qualidade?
2. A Inteligência Artificial avalia produtos da mesma forma que humanos?
3. O que os clientes mais valorizam na hora da compra?

**O Diferencial:** Além da análise estatística, utilizamos técnicas de **NLP** para ler os resumos gerados por IA e extrair insights de negócio para o time de Marketing.

### 📥 Etapa 1: Coleta e Carga de Dados (Data Ingestion)
Todo projeto de Data Science começa com a aquisição dos dados.
Nesta etapa, realizamos:

1.  **Conexão via API:** Utilizamos a biblioteca `kagglehub` para baixar o dataset *"Amazon Reviews and AI Narratives"* diretamente da fonte, garantindo que estamos usando a versão mais atualizada.
2.  **Leitura do Arquivo:** Identificamos o arquivo principal (`final_narratives.csv`) dentro da pasta baixada e o carregamos para um DataFrame do Pandas.
3.  **Inspeção Inicial:** Exibimos as primeiras linhas (`df.head()`) para entender a estrutura da tabela e verificar se as colunas (como `review_rating`, `actual_price`) foram importadas corretamente.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nadaarfaoui/amazon-reviews-and-ai-narratives-dataset")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import os

# 1. Definir o caminho onde os arquivos estão (baseado no seu print)
path = "/kaggle/input/amazon-reviews-and-ai-narratives-dataset"

# 2. Carregar o arquivo principal (o "final_narratives.csv" é o mais limpo para começar)
# Vamos listar os arquivos na pasta para garantir o nome exato
arquivos = os.listdir(path)
print(f"Arquivos encontrados: {arquivos}")

# Carregando o dataset curado
df = pd.read_csv(f"{path}/final_narratives.csv")

# 3. Visualizar as primeiras linhas e informações gerais
print("\n--- Primeiras 5 linhas ---")
display(df.head())

print("\n--- Informações do Dataset ---")
df.info()

### 🧹 Etapa 2: Tratamento e Limpeza (Data Cleaning)
Ao inspecionar os dados, identificamos um problema comum em datasets reais: colunas numéricas (como Preço e Avaliação) foram interpretadas como Texto (`object`) devido à presença de caracteres especiais (símbolos de moeda, vírgulas, etc.).

**O que este código faz:**
1.  **Seleção:** Identificamos as colunas problemáticas (`discount_price`, `actual_price`, etc.).
2.  **Expressão Regular (Regex):** Utilizamos o padrão `r'[^\d.]'` para varrer cada linha e **apagar** tudo o que não for número (dígito) ou ponto decimal. Isso remove moedas e textos indesejados.
3.  **Conversão de Tipo (Casting):** Transformamos o resultado limpo em números reais (`float`), permitindo que façamos cálculos matemáticos e estatísticos nas próximas etapas.

In [ ]:
# Lista das colunas que deveriam ser números mas estão como texto
colunas_numericas = ['discount_price', 'actual_price', 'review_rating', 'no_of_ratings']

# Vamos limpar cada uma delas
for col in colunas_numericas:
    # 1. Força virar texto (str) para garantir
    # 2. Substitui tudo que NÃO for número ou ponto (.) por vazio
    # O regex r'[^\d.]' significa: "apague tudo que não for dígito ou ponto"
    df[col] = df[col].astype(str).str.replace(r'[^\d.]', '', regex=True)

    # 3. Converte para número (float). Se der erro, coloca NaN (nulo)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Vamos verificar se mudou de 'object' para 'float64'
print("--- Novos Tipos de Dados ---")
df.info()

# Vamos ver as estatísticas (Média, Máximo, Mínimo) para ver se faz sentido
print("\n--- Estatísticas Descritivas ---")
display(df.describe())

In [ ]:
# Taxa de conversão aproximada (1 INR = 0.07 BRL)
taxa_conversao = 0.07

# Criando colunas novas em Reais
df['preco_real'] = df['actual_price'] * taxa_conversao
df['desconto_real'] = df['discount_price'] * taxa_conversao

# Vamos ver como ficou
print("--- Comparação Rúpia vs Real ---")
display(df[['actual_price', 'preco_real']].head())

print("\n--- Estatísticas em Reais ---")
display(df['preco_real'].describe())

### 🔍 Etapa 3: Análise Exploratória (EDA) - Matriz de Correlação
Com os dados limpos e numéricos, entramos na análise estatística.
O objetivo aqui é medir a força da relação linear entre diferentes variáveis. Para isso, utilizamos a **Correlação de Pearson**, que varia de -1 a 1.

**O que o código faz:**
1.  **Filtro:** Selecionamos apenas as colunas numéricas relevantes para o negócio (`Preço`, `Desconto`, `Nota do Cliente`, `Nota da IA`).
2.  **Cálculo:** O método `.corr()` processa milhares de linhas e calcula como uma variável se comporta em relação à outra.
3.  **Visualização:** Usamos a biblioteca **Seaborn** para gerar um *Heatmap*.
    * **Vermelho (perto de 1.0):** Correlação Positiva Forte (quando um sobe, o outro sobe).
    * **Azul (perto de -1.0):** Correlação Negativa (quando um sobe, o outro desce).
    * **Claro (perto de 0.0):** Nenhuma relação aparente.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Configurando o tamanho da imagem
plt.figure(figsize=(10, 6))

# Selecionando apenas as colunas numéricas para correlação
# Vamos ver a relação entre: Preço, Desconto, Nota do Cliente e Nota da IA
cols_interesse = ['actual_price', 'discount_price', 'review_rating', 'best_avg_score']
correlacao = df[cols_interesse].corr()

# Criando o Mapa de Calor (Heatmap)
sns.heatmap(correlacao, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

plt.title("Mapa de Correlação: Preço x Avaliação x Nota da IA")
plt.show()

### 1. Dinheiro não compra satisfação (Preço vs. Nota)
**Observação:** Ao analisar a correlação entre `preco_real` e `review_rating`, o valor encontrado foi próximo de **0.01** (cor muito clara no mapa).

**Conclusão de Negócio:**
Identificamos que **não existe correlação linear** entre o preço do produto e a nota dada pelo cliente.
* **O que isso significa:** Um produto custar 10x mais não garante que ele terá uma nota melhor. Na Amazon, a satisfação do cliente está ligada à entrega de valor (custo-benefício) e não ao preço absoluto. Produtos baratos podem ser tão bem avaliados quanto itens de luxo, desde que cumpram o que prometem.

### 🇧🇷 Etapa 4: Localização e Engenharia de Atributos (Feature Engineering)
Para que a análise gere valor para stakeholders brasileiros, precisamos adaptar os dados ao nosso contexto.
Nesta etapa, realizamos duas transformações cruciais:

1.  **Conversão Monetária (Currency Conversion):**
    * Os preços originais estavam em Rúpias Indianas (INR).
    * Aplicamos uma taxa de conversão aproximada (`0.07`) para criar colunas em Reais (BRL), permitindo uma noção intuitiva de valor (saber se o produto custa R$ 50 ou R$ 5.000).

2.  **Tradução de Colunas (Renaming):**
    * Renomeamos as variáveis técnicas do inglês (ex: `actual_price`, `review_rating`) para termos de negócio em português (ex: `preco_original`, `nota_cliente`).
    * Isso não é apenas estética; é **acessibilidade**. Facilita a leitura de gráficos e relatórios por qualquer pessoa da empresa, não apenas programadores.

In [ ]:
# --- ETAPA 1: CONVERSÃO DE MOEDA (Matemática) ---
# Taxa aproximada: 1 Rúpia (INR) = 0.07 Reais (BRL)
# Vamos multiplicar as colunas de preço por 0.07
df['actual_price'] = df['actual_price'] * 0.07
df['discount_price'] = df['discount_price'] * 0.07

# --- ETAPA 2: TRADUÇÃO DAS COLUNAS (Renomeação) ---
# Dicionário DE (Inglês) -> PARA (Português)
dicionario_colunas = {
    'name': 'nome_produto',
    'main_category': 'categoria_principal',
    'sub_category': 'sub_categoria',
    'review_rating': 'nota_cliente',
    'no_of_ratings': 'num_avaliacoes',
    'discount_price': 'preco_com_desconto',  # Preço que o cliente paga
    'actual_price': 'preco_original',        # Preço "de tabela" (riscado)
    'best_model': 'melhor_modelo_ia',
    'raw_best_summary': 'resumo_ia',
    'best_avg_score': 'nota_ia'
}

# Aplicando a renomeação
df = df.rename(columns=dicionario_colunas)

# --- VERIFICANDO O RESULTADO ---
print("--- Novas Colunas em Português ---")
display(df.head(3))

print("\n--- Estatísticas em Reais (R$) ---")
# Formatando para aparecer com 2 casas decimais
display(df[['preco_original', 'preco_com_desconto']].describe().round(2))

### 📊 Etapa 5: Visualização da Correlação (Dados Localizados)
Com as colunas traduzidas e os valores convertidos para Reais, geramos o gráfico definitivo para a análise.

**Por que refazer o gráfico?**
1.  **Acessibilidade:** Ao usar eixos em português (`preco_original`, `nota_cliente`), eliminamos a barreira do idioma. O gráfico deixa de ser apenas técnico e passa a ser uma ferramenta de comunicação de negócio.
2.  **Validação de Integridade:** Observe que os valores de correlação (ex: `0.01` entre Preço e Nota) permanecem idênticos aos dados originais. Isso prova matematicamente que nossa conversão de moeda (R$ vs ₹) foi feita corretamente e não distorceu a realidade estatística dos dados.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Selecionando as colunas com os NOVOS nomes em português
cols_luso = ['preco_original', 'preco_com_desconto', 'nota_cliente', 'nota_ia']
correlacao = df[cols_luso].corr()

# Gerando o gráfico
sns.heatmap(correlacao, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Mapa de Correlação (Dados em Português)")
plt.show()

### 1. Preço alto não garante nota alta
**Observação:**
Ao analisar a correlação entre `preco_original` e `nota_cliente`, o valor encontrado foi próximo de **0.01** (cor muito clara no mapa).

**Conclusão de Negócio:**
Identificamos que **não existe correlação linear** entre o preço do produto e a nota dada pelo cliente.
* **O que isso significa:** Um produto custar R\$ 10.000 não garante que ele terá uma nota melhor do que um de R\$ 50. Na Amazon, a satisfação do cliente está ligada à entrega de valor (custo-benefício) e não ao preço absoluto. Produtos baratos podem ser tão bem avaliados quanto itens de luxo.

### 2. Divergência entre Humanos e IA (Cliente vs. Algoritmo)
**Observação:**
Ao cruzar a nota dada pelos clientes (`nota_cliente`) com a pontuação de qualidade da narrativa gerada pela IA (`nota_ia`), encontramos uma **correlação negativa fraca de -0.18**.

**Conclusão de Negócio:**
Isso indica que a IA e os Humanos avaliam critérios diferentes:
* **O Humano (Cliente):** Avalia a **experiência de uso** (funcionalidade, entrega, satisfação pessoal).
* **A Máquina (IA):** Avalia a **riqueza técnica do texto** (detalhes, estrutura da narrativa).
* **O Insight Estratégico:** Um produto pode ser amado pelos clientes (5 estrelas) mas ter uma descrição simples que a IA não considera "rica". Portanto, sistemas de recomendação não devem confiar cegamente apenas em scores automáticos de IA, pois eles podem não refletir a satisfação real do usuário.

### 3. A Lógica dos Descontos (Validação de Dados)
**Observação:**
A correlação entre `preco_original` e `preco_com_desconto` é fortíssima: **0.96** (vermelho escuro).

**Conclusão de Negócio:**
Este resultado valida a integridade dos dados.
* **A Lógica:** É matematicamente esperado que quanto maior o preço original de um produto, maior será o valor absoluto do desconto aplicado (ex: 10% de desconto em um carro é mais dinheiro do que 10% em uma bala).
* **Por que isso é importante:** Essa correlação forte confirma que não há erros graves de cálculo na base de dados, garantindo que podemos confiar nos números para as próximas análises.

### 📊 Etapa 6: Ranking de Categorias (A Descoberta do Nicho)
Nesta etapa, nosso objetivo de negócio era responder: *"Qual categoria de produtos tem a maior satisfação média?"*

**O que o código faz:**
1.  **Agrupamento:** Utilizamos `.groupby('categoria_principal')` para juntar todos os produtos da mesma família.
2.  **Média:** Calculamos a média da `nota_cliente` para cada grupo.
3.  **Visualização:** Geramos um gráfico de barras para comparar os resultados.

**O Insight (A surpresa):**
O gráfico revelou que **100% dos dados** pertencem a uma única categoria: `tv, audio & cameras`.
* **Conclusão:** Este dataset não é de um e-commerce generalista, mas sim especializado em eletrônicos.
* **Decisão de Projeto:** Como não é possível comparar categorias diferentes, nossa análise estratégica irá pivotar para investigar **Sub-categorias** e, principalmente, a **Análise de Texto (NLP)** para encontrar diferenciais competitivos dentro desse nicho.

In [ ]:
# 1. Preparando os dados (Agrupando por Categoria e calculando a Média da Nota)
ranking_categorias = df.groupby('categoria_principal')['nota_cliente'].mean().sort_values(ascending=False).reset_index()

# 2. Criando o Gráfico
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.barplot(x='nota_cliente', y='categoria_principal', data=ranking_categorias, palette='viridis')

# 3. Ajustando o visual
plt.title('Ranking de Satisfação: Qual categoria tem as melhores notas?', fontsize=14)
plt.xlabel('Nota Média (1 a 5)')
plt.ylabel('Categoria')
plt.xlim(3.5, 5.0) # Dando um "zoom" para ver melhor as diferenças
plt.show()

# 4. Mostrando a tabela com os números exatos
display(ranking_categorias)

### 📊 Etapa 6: Ranking de Categorias (A Descoberta do Nicho)
Nesta etapa, nosso objetivo de negócio era responder: *"Qual categoria de produtos tem a maior satisfação média?"*

**O que o código faz:**
1.  **Agrupamento:** Utilizamos `.groupby('categoria_principal')` para juntar todos os produtos da mesma família.
2.  **Média:** Calculamos a média da `nota_cliente` para cada grupo.
3.  **Visualização:** Geramos um gráfico de barras para comparar os resultados.

**O Insight (A surpresa):**
O gráfico revelou que **100% dos dados** pertencem a uma única categoria: `tv, audio & cameras`.
* **Conclusão:** Este dataset não é de um e-commerce generalista, mas sim especializado em eletrônicos.
* **Decisão de Projeto:** Como não é possível comparar categorias diferentes, nossa análise estratégica irá pivotar para investigar **Sub-categorias** e, principalmente, a **Análise de Texto (NLP)** para encontrar diferenciais competitivos dentro desse nicho.

In [ ]:
# 1. Tira a prova real: Quantas categorias principais existem?
print("--- Contagem por Categoria Principal ---")
print(df['categoria_principal'].value_counts())

print("\n" + "="*50 + "\n")

# 2. Vamos para o detalhe: Análise por SUB-CATEGORIA
# Agrupando e ordenando
ranking_sub = df.groupby('sub_categoria')['nota_cliente'].mean().sort_values(ascending=False).reset_index()

# 3. Gerando o Gráfico de Barras (Top 10 para não ficar polúido)
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 7))
# Pegamos apenas as top 10 subcategorias com head(10)
sns.barplot(x='nota_cliente', y='sub_categoria', data=ranking_sub.head(10), palette='magma')

plt.title('Top 10 Sub-Categorias com Maior Satisfação', fontsize=14)
plt.xlabel('Nota Média (1 a 5)')
plt.ylabel('Sub-Categoria')
plt.xlim(3.5, 5.0) # Zoom para ver as diferenças
plt.show()

# 4. Mostrando os dados
display(ranking_sub.head(10))

In [ ]:
# 1. Vamos contar quantos modelos de IA diferentes existem
contagem_modelos = df['melhor_modelo_ia'].value_counts()

print("--- Modelos de IA encontrados ---")
print(contagem_modelos)

# 2. Se tiver mais de um modelo, vamos ver qual tem a melhor nota média (nota_ia)
if len(contagem_modelos) > 1:
    ranking_ia = df.groupby('melhor_modelo_ia')['nota_ia'].mean().sort_values(ascending=False).reset_index()

    import seaborn as sns
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 6))
    sns.barplot(x='nota_ia', y='melhor_modelo_ia', data=ranking_ia, palette='rocket')
    plt.title('Batalha das IAs: Qual modelo escreve as melhores narrativas?')
    plt.xlabel('Nota Média da IA')
    plt.xlim(0, 1) # A nota da IA geralmente vai de 0 a 1
    plt.show()
else:
    print("\n⚠️ OPA! Só tem um modelo de IA também. Me avise se aparecer essa mensagem!")

### 4. Batalha de LLMs: BART vs. T5
**Observação:**
Ao comparar a performance média dos modelos de linguagem utilizados para gerar as narrativas:
* **Modelo BART:** Atingiu uma pontuação média superior a **0.55**.
* **Modelo T5:** Ficou com uma média inferior, próxima de **0.38**.

**Conclusão Técnica:**
O modelo **BART** demonstrou ser significativamente superior ao T5 para a tarefa de sumarização de avaliações neste domínio de produtos (eletrônicos).
* **Decisão de Engenharia:** Em um cenário de implementação real (deploy), o BART seria a escolha recomendada, pois entrega resumos de maior qualidade técnica, garantindo descrições mais ricas e precisas para os clientes.

### ☁️ Etapa 7: Mineração de Texto (NLP) e Word Cloud
Como descobrimos que o dataset é focado em um nicho (eletrônicos), a análise numérica de vendas (que é parecida para todos) perde força.
É aqui que entra o **Processamento de Linguagem Natural (NLP)**.

**O que o código faz:**
1.  **Agregação de Texto:** O comando `" ".join(...)` pega todos os resumos escritos pela IA na coluna `resumo_ia` e os junta em um único texto gigante.
2.  **Limpeza (Stopwords):** Utilizamos a lista `STOPWORDS` para remover palavras que não agregam valor semântico (como "the", "is", "a", "and"), deixando apenas os termos de negócio.
3.  **Visualização:** A biblioteca `WordCloud` gera a imagem onde **tamanho = frequência**.
    * **Resultado Visual:** As palavras **"Battery"**, **"Camera"** e **"Good"** aparecem gigantes.
    * **Insight Imediato:** Isso prova visualmente que a satisfação do cliente nesta categoria depende quase exclusivamente da bateria e da câmera, guiando a estratégia de marketing.

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

# 1. Juntando todos os textos da coluna 'resumo_ia' em um único texto gigante
# (Certifique-se que o nome da coluna está certo. Se der erro, tente 'raw_best_summary')
todos_resumos = " ".join(s for s in df['resumo_ia'].dropna())

# 2. Configurando a Nuvem de Palavras
# STOPWORDS são palavras vazias (the, a, in, etc) que a gente remove para não poluir
stopwords = set(STOPWORDS)
# Adicione palavras extras se precisar (ex: "product", "amazon")
stopwords.update(["product", "amazon", "one", "use"])

wordcloud = WordCloud(stopwords=stopwords,
                      background_color="black",
                      width=1600, height=800,
                      colormap='viridis').generate(todos_resumos)

# 3. Exibindo a imagem
plt.figure(figsize=(15, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off") # Tira as bordas com números
plt.title("O que a IA mais destaca nos produtos?", fontsize=20, color='gray')
plt.show()

### 5. Nuvem de Palavras: O que o cliente valoriza?
**Observação:**
A análise visual das palavras mais frequentes nos resumos gerados pela IA (Word Cloud) revelou três pilares principais:
1. **Sentimento:** Predominância absoluta de termos positivos ("Awesome", "Good", "Great"), alinhado com a alta média de notas do dataset.
2. **Funcionalidade:** As características técnicas mais citadas foram **"Battery Life"** (Vida útil da bateria) e **"Camera"**.
3. **Marca:** A presença destacada da palavra **"Apple"** indica a relevância da marca nas avaliações analisadas.

**Conclusão de Negócio:**
Para produtos desta categoria (Eletrônicos/Smartphones), a satisfação do cliente é definida primariamente pela **autonomia da bateria** e **qualidade da câmera**.
* **Ação Recomendada:** Em campanhas de marketing para estes produtos, destacar termos como "Bateria de longa duração" e "Câmera de alta resolução" tende a gerar maior conversão, pois são as dores e desejos mais latentes nos resumos de quem já comprou.

# 🚀 Conclusão Executiva e Solução de Negócio

Após pivotar a análise para focar em Processamento de Linguagem Natural (NLP), transformamos dados técnicos em **Estratégia de Vendas**.

### 1. O Diagnóstico
A análise de correlação provou que **preço não define satisfação** (correlação 0.01). O cliente de eletrônicos é pragmático e busca funcionalidade, não luxo.

### 2. A Solução de Marketing (Actionable Insight)
A mineração de texto revelou que a satisfação do cliente está ancorada em dois pilares técnicos: **BATERIA** e **CÂMERA**.

✅ **Ação Recomendada:**
Em campanhas de marketing para estes produtos, **destacar termos como "Bateria de longa duração" e "Câmera de alta resolução" tende a gerar maior conversão.**
Os dados mostram que essas são as dores e desejos mais latentes nos resumos de quem já comprou. Focar nisso na *copy* do anúncio é atacar diretamente a necessidade do consumidor.

### 3. Decisão de Engenharia
Para a automação de resumos de produtos no site, o modelo **BART** deve ser o escolhido, pois superou o modelo T5 em qualidade técnica de escrita (Média 0.55 vs 0.38).

---
*Projeto de Portfólio de Data Science - Focado em Business Analytics.*